In [3]:
import streamlit as st#Repo
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import tweepy
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
pd.set_option('display.max_columns', None)
st.set_page_config(layout="wide")

# Twitter API credentials
consumer_key = os.environ["twtr_consumer_key"]
consumer_secret = os.environ["twtr_consumer_secret"]
access_key = os.environ["twtr_access_key"]
access_secret = os.environ["twtr_access_secret"]
quandl_api_key = os.environ["quandl_api_key"]

# TODO STORE FILE CREATINO DATE IN CSV FILENAMES TO CHECK WHEN WAS CREATED AND 24 hours limit on updatin new tweets!

def get_all_stored_twitter_user_csvs():
    # get all csv file names - already scraped users
    extension = 'csv'
    all_twitter_user_scraped_csvs = glob.glob(
        r'twitterdata/*.{}'.format(extension))  # CHANGE FOR SHARE STREAMLIT to /
    # filter the price csv
    all_twitter_user_scraped_csvs = [
        k for k in all_twitter_user_scraped_csvs if 'BITFINEX' not in k]
    # filter the price csv
    all_twitter_user_scraped_csvs = [
        k for k in all_twitter_user_scraped_csvs if 'relevant_words' not in k]

    display_name_all_twitter_user_scraped_csvs = [
        i.split(' ', 1)[0].split("twitterdata\\")[1] for i in all_twitter_user_scraped_csvs]

    return display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs



In [4]:
display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs = get_all_stored_twitter_user_csvs()

In [7]:
all_twitter_user_scraped_csvs

['twitterdata\\100trillionUSD 1303661563291807745.csv',
 'twitterdata\\100trillionUSD 1328299487396327427.csv',
 'twitterdata\\100trillionUSD 1350414544007557120.csv',
 'twitterdata\\Alice_comfy 1408971288085811200.csv',
 'twitterdata\\Alice_comfy 1411425879041867776.csv',
 'twitterdata\\Alice_comfy 1414030084919357443.csv',
 'twitterdata\\AltcoinGordon 1441359529115992079.csv',
 'twitterdata\\AltcoinGordon 1489988707683905536.csv',
 'twitterdata\\APompliano 1417913298549346310.csv',
 'twitterdata\\APompliano 1457852616906575874.csv',
 'twitterdata\\BarrySilbert 1054378082059075584.csv',
 'twitterdata\\BarrySilbert 972094439752519684.csv',
 'twitterdata\\BillyBobBaghold 1444306376054956031.csv',
 'twitterdata\\BillyBobBaghold 1474162848297324552.csv',
 'twitterdata\\BitcoinMagazine 1409942264730816518.csv',
 'twitterdata\\BitcoinMagazine 1461133227993845765.csv',
 'twitterdata\\BTCTN 1403132645475815428.csv',
 'twitterdata\\BTCTN 1459809030063415296.csv',
 'twitterdata\\coinmamba 13848

In [9]:
display_name_user_selection_list_containing_twitter_user = st.sidebar.selectbox(
    "Select existing Twitter-User", list(display_name_all_twitter_user_scraped_csvs), 0)
display_name_user_selection_list_containing_twitter_user

'100trillionUSD'

In [39]:
# map name back to filename:
user_selection_list_containing_twitter_user = [
    k for k in all_twitter_user_scraped_csvs if display_name_user_selection_list_containing_twitter_user in k]
user_selection_list_containing_twitter_user = user_selection_list_containing_twitter_user

In [40]:
user_selection_list_containing_twitter_user

['twitterdata\\100trillionUSD 1303661563291807745.csv',
 'twitterdata\\100trillionUSD 1328299487396327427.csv',
 'twitterdata\\100trillionUSD 1350414544007557120.csv']

In [41]:
# def main(user_selection_list_containing_twitter_user):
display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs = get_all_stored_twitter_user_csvs()

In [42]:
user_selection_list_containing_twitter_user

['twitterdata\\100trillionUSD 1303661563291807745.csv',
 'twitterdata\\100trillionUSD 1328299487396327427.csv',
 'twitterdata\\100trillionUSD 1350414544007557120.csv']

In [43]:
# st.write(user_selection_list_containing_twitter_user)
# st.write(all_twitter_user_scraped_csvs)
# map name back to filename:
# empty space at the end to check if its the exact same name and
# not matching until there like AltcoinGordon for AltcoinGordons
# user_selection_list_containing_twitter_user = [
#     k for k in all_twitter_user_scraped_csvs if user_selection_list_containing_twitter_user in k]

   

In [44]:
user_selection_list_containing_twitter_user

['twitterdata\\100trillionUSD 1303661563291807745.csv',
 'twitterdata\\100trillionUSD 1328299487396327427.csv',
 'twitterdata\\100trillionUSD 1350414544007557120.csv']

In [45]:

# here we can get multiple files from different times of scraping the specific user
# now we read all of those files and merge them together into one df
list_of_dfs = []
for user_file in user_selection_list_containing_twitter_user:
    tweet_data = pd.read_csv("{}".format(user_file))
    list_of_dfs.append(tweet_data)

tweet_data = pd.concat(list_of_dfs)

st.session_state.first_run = False


In [47]:
tweet_data

,id,created_at,text
0,1456985396815044615,2021-11-06 14:02:42+00:00,RT @WClementeIII: This week I sat down with bo...
1,1456900938317418497,2021-11-06 08:27:06+00:00,@Mynima4 🌹
2,1456900725548666882,2021-11-06 08:26:15+00:00,@fuserleer 🌹
3,1456898176691097602,2021-11-06 08:16:07+00:00,Movie tip for the weekend: V for Vendetta! It ...
4,1456763288721649667,2021-11-05 23:20:08+00:00,"@geoff_l @Rager I wish that were true, however..."
...,...,...,...
3171,1350416173490434050,2021-01-16 12:14:39+00:00,@novisport If tether would not be backed (whic...
3172,1350415871584460803,2021-01-16 12:13:27+00:00,@Worteltje_Pi @Polylunar_ @nic__carter All tra...
3173,1350415328912801795,2021-01-16 12:11:18+00:00,"@Polylunar_ @nic__carter Ah ok, didnt see that..."
3174,1350414889127448577,2021-01-16 12:09:33+00:00,@melkrever @Geldprinterbrrr Ja dat is wat mijn...


In [46]:
# for the right naming we take the first entry
user_selection_list_containing_twitter_user = user_selection_list_containing_twitter_user[0]
user_selection_list_containing_twitter_user

'twitterdata\\100trillionUSD 1303661563291807745.csv'

In [ ]:
 

    # streamlit layout
    st.title("Tweets on charts - {}".format(
        user_selection_list_containing_twitter_user.split(" ")[0].split("twitterdata/")[1]))

    # # # start - read in BTC data # # #
    datasource_btcusd = "BITFINEX/BTCUSD.csv"
    btcusd_data = pd.read_csv("coindata/{}".format(
        datasource_btcusd.replace("/", " ")), index_col=0)
    btcusd_data.index = pd.to_datetime(btcusd_data.index)

    most_recent_stored_btcusd_date = btcusd_data.sort_index().tail(
        1).index[0].strftime("%Y-%m-%d")
    todays_date = datetime.date.today() - timedelta(days=1)
    todays_date = todays_date.strftime("%Y-%m-%d")

    # st.write(most_recent_stored_btcusd_date)
    # st.write(todays_date)

    if most_recent_stored_btcusd_date != todays_date:
        data = q.get(datasource_btcusd.split(".")[0],   start_date=most_recent_stored_btcusd_date,
                     end_date='{}'.format(todays_date),
                     api_key=quandl_api_key)
        data.info()
        data["First"] = data.Last.shift(1)
        data.dropna()
        btcusd_data = pd.concat([btcusd_data, data])
        btcusd_data = btcusd_data.sort_index()
        # store current df with up-to-date values
        btcusd_data.to_csv('coindata/{}'.format(
            datasource_btcusd.replace("/", " ")), index=True)
    # # # end - read in BTC data # # #

    # # # start - processing and cleaning of tweets # # #
    # Instantiate the sentiment intensity analyzer
    rel_tweet_data = tweet_data[tweet_data['text'].str.contains(
        '|'.join(option))]
    # rel_tweet_data = rel_tweet_data[~rel_tweet_data['text'].str.contains("@")]

    rel_tweet_data["created_at"] = pd.to_datetime(
        rel_tweet_data["created_at"])
    rel_tweet_data['created_at'] = rel_tweet_data['created_at'].dt.tz_localize(
        None)
    rel_tweet_data['created_at_day'] = rel_tweet_data['created_at'].dt.round(
        '1d')
    rel_tweet_data_incl_price = pd.merge(
        rel_tweet_data, btcusd_data, how="left", left_on=rel_tweet_data["created_at_day"], right_on=btcusd_data.index,)
    # # # end - processing and cleaning of tweets # # #

    # # # start - chart with tweets # # #
    fig = go.Figure(
        data=[go.Candlestick(x=btcusd_data.index,
                             open=btcusd_data['First'],
                             high=btcusd_data['High'],
                             low=btcusd_data['Low'],
                             close=btcusd_data['Last'],
                             name="{}".format(datasource_btcusd.split("/")
                                              [1].split(".")[0]),
                             )],
    )

    # add tweets to the candle - chart
    fig.add_trace(
        go.Scatter(mode="markers",
                   x=rel_tweet_data_incl_price["created_at"],
                   y=rel_tweet_data_incl_price["High"]*1.1,
                   name='tweets',
                   text=rel_tweet_data_incl_price["text"],
                   textposition="top center",
                   marker={'color': "#0d75f8",  # clear blue
                           'size': 4
                           }
                   )
    )
    fig.update_layout(
        # title="Plot Title",
        autosize=False,
        width=int(1400/1.3),
        height=int(800/1.3),
        xaxis_range=[rel_tweet_data_incl_price["created_at"].min(
        ), rel_tweet_data_incl_price["created_at"].max()]
        # fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]
    )
    st.plotly_chart(fig)
    # # # end - chart with tweets # # #

    # # # start - add word cloud # # #
    # get all the rows into one string
    complete_str = rel_tweet_data['text'].str.cat(sep=' ')

    stopwords = set(STOPWORDS)
    stopwords.update(["and", "or", "https", "year", "will", "post", "see",
                      "going", "now", "actually", "co", "go",  "look", "make", "right", "people",
                      "RT", "really", "first", "right", "week", "still", "twitter",
                      "even", "re", "lol", "new", "much", "day", "haha", "dont", "well", "us", "sure",
                      "don", "pretty", "looks", "Thank", "another", "thing", "view", "lot",
                      "next", "many", "way", "one", "Oh", "say", "im"])
    wordcloud = WordCloud(width=int(1200/1.5), stopwords=stopwords, height=int(800/1.5), max_font_size=200,
                          max_words=50, collocations=False, background_color='black').generate(complete_str)
    fig = plt.figure(figsize=(40, 30))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    st.pyplot(fig)
    # # # end - add word cloud # # #


run_it = st.sidebar.button('Show visualizations')

st.sidebar.text("")


display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs = get_all_stored_twitter_user_csvs()

display_name_user_selection_list_containing_twitter_user = st.sidebar.selectbox(
    "Select existing Twitter-User", list(display_name_all_twitter_user_scraped_csvs), 0)

# map name back to filename:
user_selection_list_containing_twitter_user = [
    k for k in all_twitter_user_scraped_csvs if display_name_user_selection_list_containing_twitter_user in k]
user_selection_list_containing_twitter_user = user_selection_list_containing_twitter_user[0]

allowed_user_input_characters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D',
                                 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '$', '%', '_', '!', '§']


# add new twitter user data
twitter_name = ""

# update_selected_user = st.sidebar.button(
#     'Update tweets')

user_input_twitter_name = st.sidebar.text_input(
    "Add new Twitter-User data", twitter_name)


st.sidebar.text("")
st.sidebar.text("")

new_search_word = ""
user_input_new_search_word = st.sidebar.text_input(
    "Add new Searchword", new_search_word)

datasource = "relevant_words.csv"
relevant_words = []
with open("tmp/" + datasource, newline='') as inputfile:
    for row in csv.reader(inputfile):
        relevant_words.append(row)
relevant_words = relevant_words[0]

if user_input_new_search_word:
    button_add_new_searchword = st.sidebar.button('Add Searchword')
    if button_add_new_searchword:
        if user_input_new_search_word not in relevant_words:

            if any(x not in allowed_user_input_characters for x in user_input_new_search_word):
                st.error(
                    'Character not allowed, please dont use special characters')
            else:
                relevant_words.append(user_input_new_search_word)
                with open("tmp/" + datasource, 'w') as f:
                    write = csv.writer(f)
                    write.writerow(relevant_words)
                st.success('Searchword added')

        else:
            st.error('Searchword already there')


relevant_words = []
with open("tmp/" + datasource, newline='') as inputfile:
    for row in csv.reader(inputfile):
        relevant_words.append(row)
relevant_words = relevant_words[0]

# # # start - read in BTC data # # #
option = st.sidebar.multiselect(
    'Searchwords:', relevant_words, relevant_words
)



if run_it or st.session_state.first_run:
    main(user_selection_list_containing_twitter_user)
